In [9]:
import pandas as pd
from datetime import datetime, timedelta
pd.options.display.max_columns = 99

In [2]:
customer = pd.read_csv('./data/Customer.csv')
isin = pd.read_csv('./data/Isin.csv')
market = pd.read_csv('./data/Market.csv')
marketMacro = pd.read_csv('./data/MarketData_Macro.csv')
trade = pd.read_csv('./data/Trade.csv')
challenge = pd.read_csv('./data/Challenge_20180423.csv')
All = [customer, isin, market, marketMacro, trade, challenge]

In [3]:
for item in All:
    print (item.columns)
    print (item.shape)
    print ("\n")

Index(['CustomerIdx', 'Sector', 'Subsector', 'Region', 'Country'], dtype='object')
(3471, 5)


Index(['IsinIdx', 'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey',
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType'],
      dtype='object')
(27411, 17)


Index(['IsinIdx', 'DateKey', 'Price', 'Yield', 'ZSpread'], dtype='object')
(9867747, 5)


Index(['DateKey', 'SSE', 'DAX', 'EUROSTOXX', 'VSTOXX', 'FTSE100', 'HSI',
       'NIKKEI', 'DOWJONES_INDU', 'SP500',
       ...
       'Swap_TRY2Y', 'Swap_TRY5Y', 'Swap_USD10Y', 'Swap_USD2Y', 'Swap_USD30Y',
       'Swap_USD5Y', 'Swap_ZAR10Y', 'Swap_ZAR2Y', 'Swap_ZAR30Y', 'Swap_ZAR5Y'],
      dtype='object', length=112)
(877, 112)


Index(['TradeDateKey', 'CustomerIdx', 'IsinIdx', 'BuySell', 'NotionalEUR',
       'Price', 'TradeStatus', 'CustomerInterest'],
      dtype='object')
(6762021, 8)


In [8]:
isin.head(), trade.head(), customer.head(), market.head()

(   IsinIdx  TickerIdx  ActualMaturityDateKey  IssueDateKey Seniority Currency  \
 0        0        238               20381231      20051129       GOV      USD   
 1        1        238               20331231      20051129       GOV      USD   
 2        2        238               20331231      20051129       GOV      ARS   
 3        3        236               20170417      20070417       GOV      USD   
 4        4        234               20221004      20100222       GOV      ARS   
 
        ActivityGroup    Region   Activity RiskCaptain          Owner  \
 0  FLOW LOCAL MARKET  AMERICAS  ARGENTINA   ARGENTINA  EMK ARGENTINA   
 1  FLOW LOCAL MARKET  AMERICAS  ARGENTINA   ARGENTINA  EMK ARGENTINA   
 2  FLOW LOCAL MARKET  AMERICAS  ARGENTINA   ARGENTINA  EMK ARGENTINA   
 3  FLOW LOCAL MARKET  AMERICAS  ARGENTINA   ARGENTINA  EMK ARGENTINA   
 4  FLOW LOCAL MARKET  AMERICAS  ARGENTINA   ARGENTINA  EMK ARGENTINA   
 
   CompositeRating IndustrySector IndustrySubgroup MarketIssue  Is

In [10]:
def convertDatetime(x):
    x = str(x)
    date = x[:4] + '-' + x[4:6] + '-' + x[6:]
    return datetime.strptime(date, '%Y-%m-%d')

In [11]:
trade['date'] = trade['TradeDateKey'].apply(lambda x: convertDatetime(x))

In [12]:
trade['week'] = trade['date'].apply(lambda x: x - timedelta(days=x.weekday()))

In [13]:
trade = trade.drop(['TradeStatus', 'TradeDateKey', 'NotionalEUR', 'date', 'Price'], axis=1)

In [14]:
tradebyweek = trade.groupby(by=['week', 'CustomerIdx', 'IsinIdx', 'BuySell'])['CustomerInterest'].mean().reset_index()

In [17]:
tradebyweek['CustomerInterest'] = tradebyweek['CustomerInterest'].apply(lambda x: 1. if x > 0 else 0.)

In [43]:
# 1) join with isinidx
byweek = tradebyweek.merge(isin, on = 'IsinIdx', how='left')

In [45]:
# 2) join with customer
byweek = byweek.merge(customer, on='CustomerIdx', how='left')

In [21]:
market.head()

,IsinIdx,DateKey,Price,Yield,ZSpread
0,1,20160101,104.250,7.835,5.505
1,7,20160101,107.500,7.520,5.541
2,102,20160101,100.746,4.048,2.085
3,331,20160101,112.790,-0.752,-0.215
4,345,20160101,113.383,-0.667,-0.272


In [22]:
market['date'] = market['DateKey'].apply(lambda x: convertDatetime(x))

In [24]:
market['week'] = market['date'].apply(lambda x: x - timedelta(days=x.weekday()))

In [25]:
marketbyweek = market.drop(['DateKey','date'], axis=1)

In [28]:
marketgrouped = marketbyweek.groupby(by=['week','IsinIdx'])['Price','Yield','ZSpread'].mean().reset_index()

In [32]:
# convert week column: - 7 days for lag
marketgrouped['week'] = marketgrouped['week'].apply(lambda x: x - timedelta(days=7))

In [34]:
byweek.head()

,week,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_trade,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_customer,Country
0,2015-12-28,10,312,Buy,0.0,238.0,20381231.0,20051129.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN,Asset Managers & Hedge Funds,Independent Asset Manager,Asia Pacific,HONG KONG
1,2015-12-28,10,316,Sell,0.0,238.0,20331231.0,20051129.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,Americas,BRAZIL
2,2015-12-28,10,2686,Sell,0.0,238.0,20331231.0,20051129.0,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.501245e+10,FIXED,Asset Managers & Hedge Funds,Hedge Fund,Americas,BRAZIL
3,2015-12-28,10,4305,Buy,0.0,236.0,20170417.0,20070417.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,B,Government,Sovereign,Domestic,7.340076e+09,FIXED,Asset Managers & Hedge Funds,NaN,"Europe, Middle East and Africa",GERMANY
4,2015-12-28,10,4306,Sell,0.0,234.0,20221004.0,20100222.0,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,3.058452e+09,FLOATING,Asset Owners,Insurance,"Europe, Middle East and Africa",GERMANY


In [47]:
# 3) join with market
combined = byweek.merge(marketgrouped,on =['week','IsinIdx'], how='left')

In [54]:
combined.head()

,week,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_x,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_y,Country,Price,Yield,ZSpread
0,2015-12-28,10,312,Buy,0.0,3197,20280301,20171213,SEN,USD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY YANKEE,BB-,Communications,Cable/Satellite TV,Priv placement,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
1,2015-12-28,10,316,Sell,0.0,3099,20201125,20131125,SEN,CAD,FLOW G10,AMERICAS,CDS AND HY,CDS AND HY,US HY FIN AUTOS,BBB+,Communications,Telephone-Integrated,Global,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,NaN,NaN,NaN
2,2015-12-28,10,2686,Sell,0.0,722,20170619,20070619,SEN,USD,SAS & COVERED BONDS,USD SAS,USD SAS,USD SAS,USD SAS,AAA,Financial,Mortgage Banks,Euro mtn,1.000000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,106.0530,1.3598,0.4164
3,2015-12-28,10,4305,Buy,0.0,24,20250728,20150728,SUB,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN LT2,BBB,Financial,Commer Banks Non-US,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,100.1372,4.7316,2.7202
4,2015-12-28,10,4306,Sell,0.0,24,20170202,20120202,SEN,USD,FLOW G10,AMERICAS,HG CASH,HG CASH FIN,YANKEES FIN SHORT CASH,A,Financial,Money Center Banks,Priv placement,1.500000e+09,FIXED,Corporation,Corp - Consumer Staples,Americas,USA,102.8962,1.4870,0.6444


In [56]:
len(combined)

6262730

In [53]:
combined.columns

Index(['week', 'CustomerIdx', 'IsinIdx', 'BuySell', 'CustomerInterest',
       'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey', 'Seniority',
       'Currency', 'ActivityGroup', 'Region_x', 'Activity', 'RiskCaptain',
       'Owner', 'CompositeRating', 'IndustrySector', 'IndustrySubgroup',
       'MarketIssue', 'IssuedAmount', 'CouponType', 'Sector', 'Subsector',
       'Region_y', 'Country', 'Price', 'Yield', 'ZSpread'],
      dtype='object')

In [55]:
combined.to_csv('./data/byweek_lag.csv', index=False)

In [ ]:
keep_cols = ['week', 'CustomerIdx', 'IsinIdx', 'BuySell', 'ActualMaturityDateKey', 'Seniority',
            'Activity', 'CompositeRating', 'IndustrySubgroup', 'MarketIssue']

In [52]:
combined_small = combined[['week', 'CustomerIdx', 'IsinIdx', 'BuySell', 'ActualMaturityDateKey', 'Se']]

,week,CustomerIdx,IsinIdx,BuySell,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_x,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_y,Country,Price,Yield,ZSpread
24703,2015-12-28,178,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,Americas,USA,103.9,7.8706,5.6304
34994,2015-12-28,247,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Independent Asset Manager,"Europe, Middle East and Africa",UNITED KINGDOM,103.9,7.8706,5.6304
77792,2015-12-28,650,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,"Europe, Middle East and Africa",NETHERLANDS,103.9,7.8706,5.6304
109208,2015-12-28,761,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,"Europe, Middle East and Africa",FRANCE,103.9,7.8706,5.6304
156951,2015-12-28,993,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,"Europe, Middle East and Africa",UNITED KINGDOM,103.9,7.8706,5.6304
160375,2015-12-28,1008,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Banks and Intermediaries,Bank,"Europe, Middle East and Africa",UNITED KINGDOM,103.9,7.8706,5.6304
194160,2015-12-28,1266,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,NaN,"Europe, Middle East and Africa",LUXEMBOURG,103.9,7.8706,5.6304
241634,2015-12-28,1588,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Independent Asset Manager,Americas,USA,103.9,7.8706,5.6304
245705,2015-12-28,1601,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Hedge Fund,Americas,USA,103.9,7.8706,5.6304
311664,2015-12-28,1856,1,Sell,0.0,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Independent Asset Manager,Americas,USA,103.9,7.8706,5.6304
